In [135]:
import numpy as np
import pandas as pd
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

In [91]:
first_import = pd.read_csv('./clean_frame.csv',delimiter='\t', low_memory=False)

In [92]:
print first_import

     Unnamed: 0  user_id  gender_yr4  citizen_yr4  workhours_yr4  \
0             0     -1.0         1.0         -1.0            5.0   
1             1      2.0         1.0          1.0            4.0   
2             2      4.0         1.0          1.0            6.0   
3             3      5.0        -1.0          1.0            4.0   
4             4      7.0         1.0          1.0            4.0   
5             5      8.0         1.0          1.0            4.0   
6             6      9.0         1.0          1.0            2.0   
7             7     10.0         1.0          1.0            1.0   
8             8     11.0         1.0          1.0            1.0   
9             9     12.0         1.0         -1.0            6.0   
10           10     13.0         1.0          1.0            5.0   
11           11     14.0         1.0          1.0            5.0   
12           12     15.0         1.0         -1.0            4.0   
13           13     16.0         1.0          1.

In [93]:
first_import = first_import.drop(['Unnamed: 0'],axis=1)
print first_import.columns

Index([u'user_id', u'gender_yr4', u'citizen_yr4', u'workhours_yr4',
       u'tradeexp_yr4', u'tradeenjoy_yr4', u'newshrs_yr4', u'raven_score_yr4',
       u'crt_score_yr4', u'crt_1_cor_yr4',
       ...
       u'motivation_2', u'motivation_3', u'motivation_4', u'motivation_5',
       u'motivation_6', u'motivation_7', u'motivation_8',
       u'motivation_intrinsic_score', u'motivation_prosocial_score',
       u'motivation_extrinsic_score'],
      dtype='object', length=168)


In [94]:
print first_import['newshrs_yr4'].value_counts()

 10.00    142
 5.00     107
 3.00      73
 2.00      70
 4.00      66
 7.00      57
 15.00     51
 8.00      51
 6.00      46
 20.00     40
 1.00      30
 14.00     28
 12.00     22
 2.50      18
 1.50      13
 3.50      12
 30.00      9
 25.00      8
 0.00       8
 4.50       6
 7.50       5
 0.50       5
 35.00      5
 10.50      4
 21.00      4
 28.00      3
 50.00      3
 16.00      3
-1.00       3
 9.00       2
 13.00      2
 40.00      2
 18.00      2
 22.00      2
 23.00      1
 6.50       1
 42.00      1
 12.50      1
 8.50       1
 80.00      1
 11.00      1
 45.00      1
 5.50       1
 32.00      1
 13.50      1
 0.25       1
 3.75       1
Name: newshrs_yr4, dtype: int64


In [95]:
print first_import.dtypes

user_id                       float64
gender_yr4                    float64
citizen_yr4                   float64
workhours_yr4                 float64
tradeexp_yr4                  float64
tradeenjoy_yr4                float64
newshrs_yr4                   float64
raven_score_yr4               float64
crt_score_yr4                 float64
crt_1_cor_yr4                 float64
crt_2_cor_yr4                 float64
crt_3_cor_yr4                 float64
crt_4_cor_yr4                 float64
crt_5_cor_yr4                 float64
crt_6_cor_yr4                 float64
crt_7_cor_yr4                 float64
crt_8_cor_yr4                 float64
crt_9_cor_yr4                 float64
crt_10_cor_yr4                float64
crt_11_cor_yr4                float64
crt_12_cor_yr4                float64
crt_13_cor_yr4                float64
crt_14_cor_yr4                float64
crt_15_cor_yr4                float64
crt_16_cor_yr4                float64
crt_17_cor_yr4                float64
crt_18_cor_y

In [96]:
g = first_import.columns.to_series().groupby(first_import.dtypes).groups
print g

{dtype('float64'): Index([u'user_id', u'gender_yr4', u'citizen_yr4', u'workhours_yr4',
       u'tradeexp_yr4', u'tradeenjoy_yr4', u'newshrs_yr4', u'raven_score_yr4',
       u'crt_score_yr4', u'crt_1_cor_yr4',
       ...
       u'motivation_2', u'motivation_3', u'motivation_4', u'motivation_5',
       u'motivation_6', u'motivation_7', u'motivation_8',
       u'motivation_intrinsic_score', u'motivation_prosocial_score',
       u'motivation_extrinsic_score'],
      dtype='object', length=168)}


In [97]:
user_id_latent_vars = pd.read_csv('latent_vectors.csv', header=None,delimiter=' ', low_memory=False)

In [101]:
user_id_latent_vars.insert(0, 'user_id', range(len(user_id_latent_vars)))
print user_id_latent_vars

      user_id         0         1         2         3         4
0           0  0.801119  0.282122 -0.069982  0.088061  0.012564
1           1  0.281693  0.140392  0.035321  0.071278  0.116399
2           2 -1.426946 -0.496993 -0.154872 -0.658918 -1.117059
3           3  1.555524 -1.041844 -1.051234 -0.784799  0.045633
4           4 -0.658844  1.023427 -0.920273 -0.071181  0.031747
5           5  0.047146  0.004539  0.168807  0.178217  0.135203
6           6 -0.265250  0.116643 -0.696398  0.972239  0.119027
7           7  0.032499  0.018015  0.555362  1.665917  0.097454
8           8 -0.142926 -0.267900 -0.363851  0.290011  0.123941
9           9 -0.187400  1.656197 -0.506597  0.848599  0.037224
10         10 -0.699145 -0.339964  1.302983  0.381225  0.067343
11         11 -1.342690  1.005448 -0.288503  0.545845  0.048617
12         12 -0.194383 -0.017577  0.327792  0.117514  0.122522
13         13 -0.097314  1.063165 -0.333109 -0.319708  0.089516
14         14  1.702321 -1.458340  0.209

In [102]:
merged_data = pd.merge(user_id_latent_vars,first_import,on='user_id')

In [90]:
print merged_data

     user_id         0         1         2         3         4  gender_yr4  \
0          2 -1.426946 -0.496993 -0.154872 -0.658918 -1.117059         1.0   
1          4 -0.658844  1.023427 -0.920273 -0.071181  0.031747         1.0   
2          5  0.047146  0.004539  0.168807  0.178217  0.135203        -1.0   
3          7  0.032499  0.018015  0.555362  1.665917  0.097454         1.0   
4          8 -0.142926 -0.267900 -0.363851  0.290011  0.123941         1.0   
5          9 -0.187400  1.656197 -0.506597  0.848599  0.037224         1.0   
6         10 -0.699145 -0.339964  1.302983  0.381225  0.067343         1.0   
7         11 -1.342690  1.005448 -0.288503  0.545845  0.048617         1.0   
8         12 -0.194383 -0.017577  0.327792  0.117514  0.122522         1.0   
9         13 -0.097314  1.063165 -0.333109 -0.319708  0.089516         1.0   
10        14  1.702321 -1.458340  0.209531  0.314762  0.053497         1.0   
11        15 -0.075937 -0.086714  0.522809  0.384824  0.120009  

In [103]:
np_merged = merged_data.values

In [105]:
print np_merged.shape

(914, 173)


In [106]:
def build_model(input_dim, mid_dimension, output_dim):
    model = torch.nn.Sequential()
    # computes w_c^T x + b_c 
    model.add_module("linear1",
                     torch.nn.Linear(input_dim, mid_dimension))
    model.add_module("softmax1", torch.nn.LogSoftmax())
    model.add_module("linear2",
                     torch.nn.Linear(mid_dimension, output_dim))
    # NEEDS TO BE MODIFIED FOR REGRESSION HERE
    # Compute our log softmax term.
    #model.add_module("softmax2", torch.nn.LogSoftmax())
    return model

In [108]:
def NLLLossForNN(y_pred,target):
    loss = nn.NLLLoss(size_average=True)
    loss = loss.forward(y_pred,target)
    return loss

In [110]:
def train(model, loss, optimizer, x_val, y_val):

    # Resets the gradients to 0
    optimizer.zero_grad()

    # Computes the forward pass
    fx = model.forward(x_val)

    # Computes a loss. Gives a scalar. 
    #output = custom_loss(fx,y_val,lam,model)
    output = loss.forward(fx, y_val)

    # Magically computes the gradients. 
    output.backward()

    # updates the weights
    optimizer.step()
    return output.data[0]

In [ ]:
def predict(model, x_val):
    x = Variable(x_val, requires_grad=False)
    output = model.forward(x)
    return output.data.numpy().argmax(axis=1)

def validate_performance(test_model, test_iter, batch_size, text_field):
    batch_size = 100
    n_examples=1000
    num_batches = n_examples // batch_size
    num_correct = 0
    for batch in test_iter:
        x = bag_of_words(batch, text_field)
        y = batch.label - 1
        x = x.type(torch.FloatTensor)
        predY = predict(test_model, x.data)
        num_correct += (predY == y.data.numpy()).sum()
    print num_correct
    return float(num_correct) / n_examples

In [140]:
torch_data_set = Variable(torch.from_numpy(np_merged)).float()
print torch_data_set

Variable containing:
   2.0000   -1.4269   -0.4970  ...     5.0000    5.0000    6.0000
   4.0000   -0.6588    1.0234  ...     5.0000    4.0000    2.0000
   5.0000    0.0471    0.0045  ...     5.0000    5.0000    2.0000
             ...                ⋱                ...             
 997.0000   -2.0550    0.3371  ...    -1.0000   -1.0000   -1.0000
 998.0000   -1.7399   -1.1029  ...    -1.0000   -1.0000   -1.0000
 999.0000    0.2361    0.6687  ...    -1.0000   -1.0000   -1.0000
[torch.FloatTensor of size 914x173]



In [141]:
Y = torch_data_set[:,1:6]
print Y

Variable containing:
-1.4269 -0.4970 -0.1549 -0.6589 -1.1171
-0.6588  1.0234 -0.9203 -0.0712  0.0317
 0.0471  0.0045  0.1688  0.1782  0.1352
                   ⋮                    
-2.0550  0.3371 -1.1760 -0.8284 -0.0661
-1.7399 -1.1029 -0.9105  0.0037  0.0627
 0.2361  0.6687 -0.0587  0.7734  0.0467
[torch.FloatTensor of size 914x5]



In [142]:
X = torch_data_set[:,6:]
print X

Variable containing:
  1.0000   1.0000   4.0000  ...    5.0000   5.0000   6.0000
  1.0000   1.0000   6.0000  ...    5.0000   4.0000   2.0000
 -1.0000   1.0000   4.0000  ...    5.0000   5.0000   2.0000
           ...               ⋱              ...            
  1.0000   1.0000   1.0000  ...   -1.0000  -1.0000  -1.0000
  1.0000   1.0000   1.0000  ...   -1.0000  -1.0000  -1.0000
 -1.0000   1.0000   2.0000  ...   -1.0000  -1.0000  -1.0000
[torch.FloatTensor of size 914x167]



In [151]:
n_input_features = 167
model = build_model(167, 20, 5)

X_train = X[:800,:]
Y_train = Y[:800,:]
X_test = X[800:,:]
Y_test = Y[800:,:]
# Loss here is negative log-likelihood 
loss = torch.nn.MSELoss(size_average=True)

# Optimizer. SGD stochastic gradient. 
#optimizer = optim.SGD(model.parameters(), lr=0.001)
optimizer = optim.Adam(model.parameters(), lr=0.005)

for i in range(1000):
    optimizer.zero_grad()
    cost = 0.
    predictions = model.forward(X_train)
    output = loss(predictions,Y_train)
    if (i% 10 == 0):
        print output
    output.backward()
    optimizer.step()

test_predictions = model.forward(X_test)
test_loss = loss(test_predictions,Y_test)
print "test loss: {}".format(test_loss)
    

Variable containing:
 6.8968
[torch.FloatTensor of size 1]

Variable containing:
 3.9962
[torch.FloatTensor of size 1]

Variable containing:
 3.4587
[torch.FloatTensor of size 1]

Variable containing:
 3.2748
[torch.FloatTensor of size 1]

Variable containing:
 3.2379
[torch.FloatTensor of size 1]

Variable containing:
 3.2166
[torch.FloatTensor of size 1]

Variable containing:
 3.1932
[torch.FloatTensor of size 1]

Variable containing:
 3.1763
[torch.FloatTensor of size 1]

Variable containing:
 3.1659
[torch.FloatTensor of size 1]

Variable containing:
 3.1583
[torch.FloatTensor of size 1]

Variable containing:
 3.1521
[torch.FloatTensor of size 1]

Variable containing:
 3.1470
[torch.FloatTensor of size 1]

Variable containing:
 3.1426
[torch.FloatTensor of size 1]

Variable containing:
 3.1388
[torch.FloatTensor of size 1]

Variable containing:
 3.1354
[torch.FloatTensor of size 1]

Variable containing:
 3.1324
[torch.FloatTensor of size 1]

Variable containing:
 3.1297
[torch.Floa